In [185]:
import pandas

from datetime import datetime

from pandas import Timestamp

from pandas.tseries.offsets import DateOffset
# import zip
rateios_celesc = pandas.read_csv(r"C:\Users\LuanNogs\Documents\COMPILADO-CELESC\COMPILADO RATEIOS - CELESC.csv")

rateios_celesc.drop(["OBSERVAÇÃO"],axis=1, inplace=True)

rateios_celesc['PORCENTAGEM']=rateios_celesc['PORCENTAGEM'].apply(lambda x : float(x.replace(",",".").replace("%",""))/100)

rateios_celesc['BENEFICIÁRIA']=rateios_celesc['BENEFICIÁRIA'].astype(str)

rateios_celesc.rename({"GERADORA":"UC"},axis=1,inplace=True)


rateios_celesc['UC']=rateios_celesc['UC'].apply(str)

rateios_celesc["datetime"]=rateios_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

injecao_geradores = pandas.read_csv(r"C:\Users\LuanNogs\Documents\COMPILADO-CELESC\Faturas PLIN - CELESC.csv")

injecao_geradores.rename({"ucGeradora":"UC","referencia":"REFERÊNCIA"},axis=1,inplace=True)

columns_maintain = ["UC", "REFERÊNCIA", "INJEÇÃO TP"]

injecao_geradores = injecao_geradores[columns_maintain]

injecao_geradores["INJEÇÃO TP"]=injecao_geradores["INJEÇÃO TP"].apply(lambda x : float(x)) 

injecao_geradores["UC"]=injecao_geradores["UC"].apply(str) 

injecao_geradores["datetime"]=injecao_geradores["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

leituras_celesc = pandas.read_csv(r"C:\Users\LuanNogs\Documents\COMPILADO-CELESC\LEITURAS__Geradorasdeenergia_CELESC_Conta Corrente_Ativo_Erro de leitura_Vencidas_05-08-2024.csv")

leituras_celesc=leituras_celesc.rename({"uc":"UC","date_ref":"REFERÊNCIA","kwh_compensado":"COMPENSADO","kwh_consumed":"CONSUMIDO"},axis=1)

leituras_celesc["COMPENSADO"]=leituras_celesc["COMPENSADO"].apply(lambda x : float(abs(float(x))))

leituras_celesc.rename({"UC":"BENEFICIÁRIA"},axis = 1 ,inplace=True)

columns_maintain=["BENEFICIÁRIA","REFERÊNCIA","COMPENSADO","CONSUMIDO"]

# TRATAMENTO DE DADOS LEITURAS

leituras_celesc=leituras_celesc[columns_maintain]

leituras_celesc.rename({"UC":"BENEFICIÁRIA"},inplace=True)

leituras_celesc["BENEFICIÁRIA"]=leituras_celesc["BENEFICIÁRIA"].apply(str)

leituras_celesc["REFERÊNCIA"]=leituras_celesc["REFERÊNCIA"].apply(lambda x: "01/"+x)

leituras_celesc["datetime"]=leituras_celesc["REFERÊNCIA"].apply(lambda x : datetime.strptime(x,"%d/%m/%Y"))

rateios_celesc=rateios_celesc.set_index(["UC","datetime","REFERÊNCIA"])

injecao_geradores=injecao_geradores.set_index(["UC","datetime","REFERÊNCIA"])

leituras_celesc = leituras_celesc.drop_duplicates(["BENEFICIÁRIA","REFERÊNCIA"])

leituras_celesc.set_index(['BENEFICIÁRIA',"datetime","REFERÊNCIA"],inplace=True)

# CRIAÇÃO DE LEITURAS

missing_idx = rateios_celesc.reset_index().set_index(["BENEFICIÁRIA","datetime","REFERÊNCIA"]).index.difference(leituras_celesc.index)

# Criar um DataFrame com os índices faltantes e preenchê-los com NaN ou um valor padrão
missing_rows = pandas.DataFrame(index=missing_idx, columns=leituras_celesc.columns)

missing_rows['COMPENSADO'] = 0  # ou algum valor padrão, como 0

# Adicionar as linhas faltantes ao df2
leituras_celesc = pandas.concat([leituras_celesc, missing_rows])


def injecao_cliente_usina(row):
    try:
        inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values
        inj_cliente_usina = inj_usina * row["PORCENTAGEM"]
        return float(format(inj_cliente_usina[0],".0f"))
    
    except:
        return 0
    
rateios_celesc["INJ_CLIENTE_USINA"] = rateios_celesc.apply(injecao_cliente_usina,axis=1)

rateios_celesc.sort_index()

leituras_celesc.sort_index()

injecao_geradores.sort_index()


def dict_leitura (row:pandas.Series):
    # print(row.name)
    try:
        usinas_dict = rateios_celesc.query("BENEFICIÁRIA  == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=row.name[0],ref = row.name[2]))["INJ_CLIENTE_USINA"].droplevel('REFERÊNCIA').to_dict()
        return usinas_dict
    except: 
        # new_row = pandas.DataFrame({(row.name[0],row.name[1]-DateOffset(months=1),datetime.strftime(datetime.strptime(row.name[2],"%d/%m/%Y")-DateOffset(months=1),"%d/%m/%Y")):[0]})
        return 0    

leituras_celesc["INJEÇÃO_USINAS"] = leituras_celesc.apply(dict_leitura,axis=1)

def inj_total (row): 

    return sum(row[2].values())

leituras_celesc["INJ_TOTAL"] = leituras_celesc.apply(inj_total,axis=1)

leituras_celesc=leituras_celesc.drop(["CONSUMIDO"],axis=1)

leituras_celesc=leituras_celesc.sort_index(axis=0,level='datetime',ascending=True)

leituras_celesc.insert(3,column='SALDO USINAS',value=0)

leituras_celesc.insert(3,column='SALDO TOTAL',value=0)

leituras_celesc.insert(3,column='COMPENSADO USINAS',value=0)

# def saldo_anterior_total(row):

#     try:
#         saldo_anterior = leituras_celesc.query("BENEFICIÁRIA == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=row.name[0],ref=datetime.strftime(row.name[1]-DateOffset(months=1),"%d/%m/%Y")))['SALDO TOTAL']
#     except: 
#         saldo_anterior=0
    
#     saldo_total = saldo_anterior + row[2] - row[0]


#     return saldo_total


# leituras_celesc["SALDO TOTAL"]=leituras_celesc.apply(saldo_anterior_total,axis=1)


    


C:\Users\LuanNogs\AppData\Local\Temp\ipykernel_8664\3990114094.py:83: PerformanceWarning: indexing past lexsort depth may impact performance.
  inj_usina = injecao_geradores["INJEÇÃO TP"].loc[row.name].values


ValueError: Columns must be same length as key

In [141]:
def saldo_anterior_total(row):

    saldo_anterior = leituras_celesc.query("BENEFICIÁRIA == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=row.name[0],ref=datetime.strftime(row.name[1]-DateOffset(months=1),"%d-m-%Y")))[0]
  
    saldo_total = saldo_anterior + row[2] - row[0]

    return saldo_total


In [183]:
leituras_celesc.iloc[15][0]

0.0

In [181]:
leituras_celesc.query("BENEFICIÁRIA == '{uc}' & REFERÊNCIA == '{ref}'".format(uc=leituras_celesc.iloc[50].name[0],ref=datetime.strftime(leituras_celesc.iloc[50].name[1]-DateOffset(months=1),format="%d/%m/%Y")))["SALDO TOTAL"][0]

0